 # Train yolov3 on just kicks

The following code was originally run in google colab and was adapted from [this](https://www.youtube.com/watch?v=_FNfRtXEbr4&amp%3Bab_channel=Pysource) resource.

In [ ]:
#check if NVIDIA GPU is enabled
!nvidia-smi

Mon Nov 23 06:40:46 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#connect google colab with google drive.
from google.colab import drive
drive.mount('/content/gdrive')

!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

## Clone the Darknet repo


In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 14449 (delta 14), reused 20 (delta 12), pack-reused 14421
Receiving objects: 100% (14449/14449), 13.18 MiB | 22.60 MiB/s, done.
Resolving deltas: 100% (9820/9820), done.


## Compile Darknet using Nvidia GPU

In [ ]:
#change makefile to have GPU and OPENCV enabled
%cd darknet

!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

## Configure Darknet network for training YOLO V3

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches=4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
#create a folder on google drive t osave the weights there
!mkdir "/mydrive/yolov3"

In [ ]:
!echo "kick" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-11-22 08:40:35--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  1.26MB/s    in 2m 32s  

2020-11-22 08:43:08 (1.02 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [ ]:
#unzip my images into data/obj 
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/1GL2AXaBQFqbGNgvHrtg_seanlowkickluhkin.jpg  
  inflating: data/obj/__MACOSX/._1GL2AXaBQFqbGNgvHrtg_seanlowkickluhkin.jpg  
  inflating: data/obj/1GL2AXaBQFqbGNgvHrtg_seanlowkickluhkin.txt  
  inflating: data/obj/__MACOSX/._1GL2AXaBQFqbGNgvHrtg_seanlowkickluhkin.txt  
  inflating: data/obj/2cd89343203196727a0ee464faf07d07.jpg  
  inflating: data/obj/__MACOSX/._2cd89343203196727a0ee464faf07d07.jpg  
  inflating: data/obj/2cd89343203196727a0ee464faf07d07.txt  
  inflating: data/obj/2oxQA3ypRo2CZNS8GQ5g_QQdRYsiDQ6KDxawHETyz_muaythailowkickFB-.jpg  
  inflating: data/obj/__MACOSX/._2oxQA3ypRo2CZNS8GQ5g_QQdRYsiDQ6KDxawHETyz_muaythailowkickFB-.jpg  
  inflating: data/obj/2oxQA3ypRo2CZNS8GQ5g_QQdRYsiDQ6KDxawHETyz_muaythailowkickFB-.txt  
  inflating: data/obj/3lowkickfeature-672x365.jpg  
  inflating: data/obj/__MACOSX/._3lowkickfeature-672x365.jpg  
  inflating: data/obj/3lowkickfeature-672x365.txt  
  inflating: data/obj/9ac9d2f37e12

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.620000 0.530055 0.701818 0.928962

['15', '0.620000', '0.530055', '0.701818', '0.928962']
0 data/obj/images (103).txt
0 0.620000 0.530055 0.701818 0.928962
15 0.255291 0.825195 0.203704 0.314453

['15', '0.255291', '0.825195', '0.203704', '0.314453']
1 data/obj/unnamed (6).txt
0 0.255291 0.825195 0.203704 0.314453
15 0.679825 0.579186 0.508772 0.542986

['15', '0.679825', '0.579186', '0.508772', '0.542986']
2 data/obj/image.txt
0 0.679825 0.579186 0.508772 0.542986
15 0.533333 0.520833 0.753333 0.946429

['15', '0.533333', '0.520833', '0.753333', '0.946429']
3 data/obj/images (155).txt
0 0.533333 0.520833 0.753333 0.946429
15 0.494983 0.541667 0.588629 0.916667

['15', '0.494983', '0.541667', '0.588629', '0.916667']
4 data/obj/images (147).txt
0 0.494983 0.541667 0.588629 0.916667
15 0.688333 0.526786 0.583333 0.922619

['15', '0.688333', '0.526786', '0.583333', '0.922619']
5 data/obj/images (33).txt
0 0.688333 0.526786 0.583333 0.922619
dog

[]
person

[]
cat

[]
tv

[]
car

[]
m

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/Depositphotos_250728228_xl-2015.jpg', 'data/obj/Kicking.jpg', 'data/obj/practice-kick-techniques.jpg', 'data/obj/71qdfBG9abL.jpg', 'data/obj/switch-kick-technique-in-action-960x540.jpg', 'data/obj/Knee-Kick_02.jpg', 'data/obj/images (3).jpg', 'data/obj/unnamed (1).jpg', 'data/obj/2oxQA3ypRo2CZNS8GQ5g_QQdRYsiDQ6KDxawHETyz_muaythailowkickFB-.jpg', 'data/obj/maxresdefault (2).jpg', 'data/obj/Muay-Thai-Push-Kick-Teep-520x245.jpg', 'data/obj/unnamed (2).jpg', 'data/obj/0785e6696111c5d8879a79dd0e9715eb.jpg', 'data/obj/Muay-Thai-Moves-and-Techniques (1).jpg', 'data/obj/Muay-Thai-Moves-and-Techniques.jpg', 'data/obj/Breaking-Limits-in-Muay-Thai.jpg', 'data/obj/maxresdefault (8).jpg', 'data/obj/IMG_4982.jpg', 'data/obj/maxresdefault (6).jpg', 'data/obj/Muay-Thai-Leg-Kick.jpg', 'data/obj/muay-thai-techniques.jpg', 'data/obj/hqdefault (5).jpg', 'data/obj/Senchai_Breakdown.jpg', 'data/obj/Different-attack-moves-in-Muay-Thai.jpg', 'data/obj/martial-arts-boxing-kennett-square-hockessin-ch

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

## Start training

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.882949, GIOU: 0.881342), Class: 0.999516, Obj: 0.965451, No Obj: 0.004029, .5R: 1.000000, .75R: 1.000000, count: 6, class_loss = 0.004608, iou_loss = 0.054569, total_loss = 0.059177 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.377419, GIOU: 0.278144), Class: 0.991280, Obj: 0.002287, No Obj: 0.000007, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.249004, iou_loss = 0.286636, total_loss = 0.535639 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000001, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000028, iou_loss = 0.000000, total_loss = 0.000028 
 total_bbox = 236918, rewritten_bbox = 0.002955 % 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.829605, GIOU: 0.822915), Class: 0.999529, Obj: 0.832014, No Obj: